## Import Statements

In [21]:
import gensim.downloader as api
import numpy as np
import re
import csv
import pandas as pd
import pprint
import string
import nltk
import sys
import spacy
import os
import pickle
import random     
import json

from IPython.display import HTML
from nltk.corpus import wordnet 
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize

pathToDatasets = '../datasets/'
pathToDataScripts = '../datasets/scripts/'
filePath = '../datasets/GoogleNews-vectors-negative300.bin'
# modelBeingUsed = "glove-wiki-gigaword-300"
# modelBeingUsed = "glove-wiki-gigaword-100"
# modelBeingUsed = "glove-twitter-50"
modelBeingUsed = "glove-twitter-100"
# modelBeingUsed = "glove-twitter-200"

# DATASET_FILE_NAME = "shuffled_twitter_corpus_output"
# DATASET_FILE_NAME = "airplane_tweets"
DATASET_FILE_NAME = "cleanedTweets"



sys.path.insert(0, pathToDataScripts)
from cleanDataset import tokenize_words, dataClean


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

## Downloading binaries and models


In [22]:
print("Should I reload the model?")
tstString = input()
if("no" in tstString.lower() or "n" in tstString.lower()):
    print(" didnt reload model! ")
else:
    print("loading {0}!".format(modelBeingUsed));
    fileName = "{}.pickle".format(modelBeingUsed)
    if(os.path.exists(pathToDatasets+fileName)):
        print("loading via pickle!")
        pickle_in = open(pathToDatasets+fileName, "rb")
        word_vectors = pickle.load(pickle_in);
    else:
        print("Pickle didn't exist, therefore loading model!")
        word_vectors = api.load("{0}".format(modelBeingUsed))
        print("-- Saving to pickle file for next time! --")
        pickle_out = open(pathToDatasets+fileName,"wb")
        pickle.dump(word_vectors, pickle_out)
        pickle_out.close()
        
    nltk.download('vader_lexicon')
    nltk.download('punkt')
    nltk.download('averaged_perceptron_tagger')
    nltk.download('wordnet')
    print("Model Loaded!")

setOfBoosterWords = set()
with open(pathToDatasets + "BoosterWordList.txt") as bwf:
    pd_booster = pd.read_csv(bwf, sep='\t');
    listOfBoosterWords = (pd_booster.iloc[:,0]).tolist() 
    setOfBoosterWords = set(listOfBoosterWords[:7])
print(setOfBoosterWords)



Should I reload the model?
no
 didnt reload model! 
{'definitely', 'incredibly', 'extremely', 'absolutely', 'especially', 'completely', 'overwhelmingly'}


## Global Variables and Global Objects

In [43]:
senty = SentimentIntensityAnalyzer()
vocabulary = word_vectors.vocab;

nlp = spacy.load('en')


'''
Good sets -> {100+20}
'''
TWEET_START = 200
NUM_OF_TWEETS = 100

VERBOSE_PRINTING = True
# VERBOSE_PRINTING = False

# USE_SPACY = False
USE_SPACY = True

# COLOR_PRINTING = True
COLOR_PRINTING = False

PRINT_NEUTRAL = True
# PRINT_NEUTRAL = False

# PRINT_ALL_STRINGS = True
PRINT_ALL_STRINGS = False

HANDLE_NEGATIONS = True
# HANDLE_NEGATIONS = False

HANDLE_BOOSTERS = True
# HANDLE_BOOSTERS = False

KEEP_SENSE = True
# KEEP_SENSE = False

SHOW_ALTS = 35

NUMBER_OF_NEAR = 10
MAX_SENTIMENT_HEAVY_WORDS = 6
MAX_REPLACEMENTS = 8 # increasing this has a x(NUM_OF_SENTIMENT_WORDS) effect.

PROB_ADDING_BOOSTER = 0.40 # Keep between 0 and 1, if you want all possible, use 1. If you want none, use 0


punctuation = r"\"#$%&'+-/;<=>@[\]*^_`{|}~"
LIST_OF_NEGATIONS = ["not", "no"]
BANNED_WORDS = ["fuck","bitch", "motherfucker"]

LIST_OF_EVAL_OBJECTS = []

## Class for Sentences


In [44]:
from SentenceClass import *

In [45]:
class EvaluationObj:
    def __init__(self, main, best, worst, mainSent, bestSent, worstSent, uniqueID):
        self.main  = main
        self.best = best
        self.worst = worst
        self.mainSent = mainSent
        self.bestSent = bestSent
        self.worstSent = worstSent
        self.uniqueID = uniqueID
        

## Utility Code

In [46]:
def printStrings(sentenceObj, uniqueID = -1):
    
    
    numberOfPrints = 0
    newStrings = generateHTMLObjectsFromSentence(sentenceObj)
    mainSentiment = sentenceObj.ogSentiment;
    listOfSentencesWithSentiments = []
    bestSentiment = -sys.maxsize - 1
    worstSentiment = sys.maxsize
    
    isPositiveSentence = True if(mainSentiment >= 0) else False
    
    bestSentimentString = "";
    worstSentimentString = "";
    
    if(uniqueID != -1):
        global LIST_OF_EVAL_OBJECTS
    
    numberOfSentimentHeavyWordsYet = 0;
    for ind, tSentence in enumerate(newStrings):
            alteredTweet = tSentence.getSentence()
            htmlText = tSentence.getHTML()
            numberOfPrints+=1;
            
            
            sentimentOfNewString = tSentence.getSentiment() #senty.polarity_scores(alteredTweet)['compound']
            
            thisStringPositive = True if(sentimentOfNewString >= 0) else False
            if(thisStringPositive != isPositiveSentence and KEEP_SENSE):
                continue
            
            newObj = SentenceWithSentiment(alteredTweet, sentimentOfNewString, htmlText)
            listOfSentencesWithSentiments.append(newObj)
            
            if(sentimentOfNewString >= bestSentiment):
                bestSentiment = sentimentOfNewString
                bestSentimentString = htmlText;
            elif(sentimentOfNewString < worstSentiment):
                worstSentiment = sentimentOfNewString
                worstSentimentString = htmlText;
            
            
#             if(numberOfPrints > SHOW_ALTS): break
            if(numberOfPrints > SHOW_ALTS or PRINT_ALL_STRINGS == False): continue
            if(sentimentOfNewString == mainSentiment or sentimentOfNewString == 0.0):
                if(PRINT_ALL_STRINGS and numberOfPrints <= SHOW_ALTS): displayText("{0}: {1}".format(htmlText,sentimentOfNewString),'black')
            elif(sentimentOfNewString > mainSentiment):
                if(PRINT_ALL_STRINGS and numberOfPrints <= SHOW_ALTS): displayText("{0}: {1}".format(htmlText,sentimentOfNewString),'green')
            elif(sentimentOfNewString < mainSentiment and sentimentOfNewString != 0.0):
                if(PRINT_ALL_STRINGS and numberOfPrints <= SHOW_ALTS): displayText("{0}: {1}".format(htmlText,sentimentOfNewString),'red')
                
#     if(numberOfPrints > SHOW_ALTS): print("--- More options (total: {0}) possible, but not printed ---".format(numberOfPrints));
    print("--- More options (total: {0}) possible ---".format(numberOfPrints));
    displayText("Actual Sentence: {0} :{1}".format(sentenceObj.ogSentence, sentenceObj.ogSentiment))
    if (worstSentimentString != ""): displayText("Worst Sentence: {0} : {1}".format(worstSentimentString, worstSentiment), color='red')
    if (bestSentimentString != ""): displayText("Best Sentence: {0} : {1}".format(bestSentimentString, bestSentiment), color='green') 
        
 
        
    if(worstSentimentString != "" and bestSentimentString != "" and uniqueID != -1):
#             file_writeAll.write("Inp({3}): {0}\nB({3}): {1}\nW({3}): {2}\nInSent:{4}, BSent:{5}, WSent:{6}\n\n".format(sentenceObj.ogSentence, bestSentimentString, worstSentimentString, uniqueID, sentenceObj.ogSentiment, bestSentiment, worstSentiment));
            thisEvalObj = EvaluationObj(sentenceObj.ogSentence, bestSentimentString, worstSentimentString, sentenceObj.ogSentiment, bestSentiment, worstSentiment, uniqueID);
            LIST_OF_EVAL_OBJECTS.append(thisEvalObj)
            
            print("Wrote something!")
            

    sentenceObj.addFinalSentences(listOfSentencesWithSentiments)
    return listOfSentencesWithSentiments, sentenceObj;

def cstr(s, color='black', italics=False):
    if(COLOR_PRINTING):
        if(italics):
            return cstr("{0}".format(s), color);
        return "<text style=color:{}>{}</text>".format(color, s)
    else:
        return "{}".format(s)

def displayText(text, color='black'):
    if(COLOR_PRINTING):
        display(HTML(cstr(text, color)));
        return
    print("{}".format(text));
    
    
def cleanAndTokenizeText(text):
#     text = text.lower();
    newString = ""
    for char in text:
        if char not in punctuation:
            newString += char
    text = word_tokenize(newString)
    return text;

def getPOSTags(tweet):
    if(USE_SPACY == False):
        tags = nltk.pos_tag(tweet)
        return tags;    
    tweet = ' '.join(tweet)
    doc = nlp(tweet)
    tags = [(token.text, token.tag_) for token in doc] # since the format expected is [text,tag]
    return tags

def getAntonymsAndSynonymsOfWords(word):
    if(word not in vocabulary):
        return [], []
    setOfAntonyms = set()
    setOfSynonyms = set()
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            setOfSynonyms.add(l.name().lower())
            anton = l.antonyms()
            if(anton!=[]):
                setOfAntonyms.add(anton[0].name().lower())
    if(len(setOfAntonyms) == 0):
        if(VERBOSE_PRINTING): print("No antonyms found for word {0}".format(word))
    if(len(setOfSynonyms) == 0):
        if(VERBOSE_PRINTING): print("No synonyms found for word {0}".format(word))
            
    if(KEEP_SENSE):
        return [], list(setOfSynonyms)
    else:    
        return list(setOfAntonyms), list(setOfSynonyms)


def returnReplacementsForWord(word):
    
    if(word not in vocabulary):
        print(" --- {0} not in vocabulary ---".format(word))
        return []
    possibleReplacements = [word[0] for word in word_vectors.most_similar(word,topn=NUMBER_OF_NEAR)]
    
    if(possibleReplacements == []):
        print(" --- No replacements for word {0} ---".format(word))
    antonyms,synonyms = getAntonymsAndSynonymsOfWords(word)
    
    if(antonyms != []):
        possibleReplacements.extend(antonyms)
        if(VERBOSE_PRINTING): print("Some antonyms for word {0} are {1}".format(word, antonyms[:3]))
    if(synonyms != []):
        possibleReplacements.extend(synonyms)
        if(VERBOSE_PRINTING): print("Some synonyms for word {0} are {1}".format(word, synonyms[:3]))
    return possibleReplacements
    
def posApprovedReplacements(alternativeWords, userTokens, indexOfToken):
    if(alternativeWords == []):
        return []
    tempTokens = userTokens[:]
    POSTokens = getPOSTags(tempTokens)
    validWords = []
    
    mainTag = POSTokens[indexOfToken][1]
    mainWord = userTokens[indexOfToken]
    
    for ind,word in enumerate(alternativeWords):
        if("_" in word): continue
        tempTokens[indexOfToken] = word
        posTags = getPOSTags(tempTokens)
        newTag = (posTags[indexOfToken])[1]
        if(str(newTag) == str(mainTag)):
            validWords.append(word)
    if(validWords == [] and VERBOSE_PRINTING):
        print("No POS words found for word {} with tag {}".format(mainWord, mainTag));
    return validWords
    
def editBoosterWords(sentenceObj):
    if(HANDLE_BOOSTERS == False): return sentenceObj;
    ogSentence = sentenceObj.ogSentence;
    
    tokenizedSentence = sentenceObj.getSentenceTokens()
    
    '''
        Use the indexToAlternatives dictionary to add words into the sentences, as part of the chunking process 
        The problem with this instinctively is how do you have an index "between" two variables 
        You can't really. You'd have to have a flag that says which places it's meant to go 
        You could do it _before_ the sentiment heavy word (this makes more natural sense) or 
            you could do it _after_ the word before the sentiment heavy word (this could have its own advantages) 
    '''
    
    posTagsForWords = getPOSTags(tokenizedSentence)    
    for ind,(word,tag) in enumerate(posTagsForWords):
        tmp = ' '.join(tokenizedSentence[ind:ind+2])    
        if(word in setOfBoosterWords):
            if(senty.polarity_scores(tmp)['compound'] != 0):
                sentenceObj.addAlternativesByIndex(ind, ["IGNORE_FLAG", word]);
                continue
        elif(senty.polarity_scores(word)['compound'] != 0 and ("RB" in tag or "JJ" in tag or "VB" in tag)):  # "JJ" in tag and
            newInd = ind-1 if ((ind-1) >= 0 ) else 0
            randNum = random.uniform(0,1)
            if(randNum < PROB_ADDING_BOOSTER):
                sentenceObj.addAlternativesByIndex(newInd, ["BOOSTER_FLAG"]);
#                 if(VERBOSE_PRINTING): print("Placed booster at {0} from word {1} on word {2}".format(newInd, word,tokenizedSentence[newInd] ))
    
    return sentenceObj


def generateHTMLObjectsFromSentence(sentenceObj):
    
    allSentences = sentenceObj.getFinalSentences()
    indexToAlts = sentenceObj.indexToSetOfWords;
    indexToChange = list(indexToAlts.keys());
    
    numberOfNegationsRemoved = 0;
    listOfSentenceObjs = []
    for t_sentenceObj in allSentences:
        sentence = t_sentenceObj.getSentence()
        copySentence = t_sentenceObj.getSentenceTokens()
        subIndex = 0
        addIndex = 0
        for index in sorted(indexToChange):
            if(index > len(copySentence)):  
                copySentence[len(copySentence) - index] = cstr("{0}".format(copySentence[len(copySentence) - index]), "blue", italics=True);
                continue
            else:
                if("IGNORE_FLAG" in indexToAlts[index]):
                    if(copySentence[index] in LIST_OF_NEGATIONS):
                        continue; ## Word that _should_ be removed is present.
                    else:
                        subIndex +=1
                        numberOfNegationsRemoved +=1;
                        continue;
                elif("BOOSTER_FLAG" in indexToAlts[index]):
                    addIndex +=1;

            newIndex = index + addIndex - subIndex;
            if(newIndex >= len(copySentence)):
                newIndex = len(copySentence)-1
            elif(newIndex < 0):
                newIndex = 0;
            
            
            copySentence[newIndex] = cstr("{0}".format(copySentence[newIndex]), "blue", italics=True);
        listOfSentenceObjs.append(SentenceWithHTML(sentence, ' '.join(copySentence), t_sentenceObj.getSentiment()));
    
    if(VERBOSE_PRINTING): print("Number of negations removed : {0}".format(numberOfNegationsRemoved))
    return listOfSentenceObjs
                

## Sentence Chunking and Appending

In [47]:
def helper_combine(mainList, myList):
    '''
    helper function for CombineSentenceChunks
    '''
    newList = []
    for val in myList:
        for mainVal in mainList:
#             if(VERBOSE_PRINTING): print("Combining {0} with {1}".format(' '.join(val), ' '.join(mainVal)));
            newList.append(val + mainVal);
    return newList;

def combineSentenceChunks(wholeSentence, dictOfChunks):
    '''
        Uses the helper_combine function to generate all possible combinatios and permuations of the chunks
        and any alternatives.
        
        The logic is to use the end of the sentence, apply each possible chunk from the previous key's chunks
        to every possible chunk of this key's.
        
        The helper function is used to allow us to reuse the list of alreadyGeneratedChunks and constantly
        append to them.
        
        To understand the logic better, take a look at this gist:
        https://gist.github.com/sunnyMiglani/cf85407a9e6928237b1436cc2bc95fa4
        
    '''
    reversedKeys = sorted(dictOfChunks.keys(), reverse=True)
    completeSentences = [];
    mainArr = dictOfChunks[reversedKeys[0]]
    for ind in range(1, len(reversedKeys)):
        key = reversedKeys[ind];
        mainArr = helper_combine(mainArr, dictOfChunks[key]);
        
    return mainArr;
        
def generateSentenceChunks(wholeSentence, keyToChange, nextKey, listOfMyAlternatives):
    '''
        Generates sentence chunks by iterating through the list of alternatives
        Chunking the sentence to start from current key to next key.
        This means that the sentence always goes from key 'x' to key 'y'
        
        Example:
        "I really <hate> hot chocolate, but I <prefer> hot coffee"
        Calling generateSentenceChunks will create an example sentence:
            - "<altWordForHate> hot chocolate , but I "
        
        Remember to append the first stretch of the string to the first key's chunk for proper use!
    '''
    
    newList = list(listOfMyAlternatives)
    if(keyToChange < len(wholeSentence) and keyToChange >= 0):
        newList.append(wholeSentence[keyToChange]);
    else:
        return []
    generatedSentences = []
    
        
    for myAlt in newList:
        if("IGNORE_FLAG" == myAlt):
            newSentence = wholeSentence[keyToChange+1:nextKey];
            generatedSentences.append(newSentence)
            continue
        elif("BOOSTER_FLAG" == myAlt): 
            for booster in setOfBoosterWords:
                newSentence = wholeSentence[:]
                newSentence.insert(keyToChange+1, booster)
                generatedSentences.append(newSentence[keyToChange:nextKey+1]);
        else:
            newSentence = wholeSentence[:]
            newSentence[keyToChange] = myAlt
            generatedSentences.append(newSentence[keyToChange:nextKey]);
        
    return generatedSentences
    
def returnCombinationsOfStrings(sentenceObj):
    
    indexToWordDict = sentenceObj.indexToSetOfWords;
    originalSentence = sentenceObj.ogSentence;
    tokenizedSentence = sentenceObj.getSentenceTokens() 
    reversedKeys = sorted(indexToWordDict.keys(), reverse=True)
    dictAlternatives  = {}

    sortedKeys = sorted(indexToWordDict.keys())
    sentenceChunks = {}
    htmlChunks = {}
    
    for ind in range(0,len(sortedKeys)):
        key = sortedKeys[ind]
        nextKey = sortedKeys[ind+1] if ind+1 < len(sortedKeys) else len(tokenizedSentence)
        sentenceChunks[key] = generateSentenceChunks(tokenizedSentence, key, nextKey, indexToWordDict[key])

    if(sortedKeys[0] != 0):
        newList = []
        for thislist in sentenceChunks[sortedKeys[0]]:
            newList.append(tokenizedSentence[0:sortedKeys[0]] + thislist)
        sentenceChunks[sortedKeys[0]] = newList;
        
    finalOptions = combineSentenceChunks(tokenizedSentence, sentenceChunks)
    
    finalSentences = []
    for val in finalOptions:
        sentence = ' '.join(val)
        sentiment = senty.polarity_scores(sentence)['compound']
        newSentenceObj = SentenceWithSentiment(sentence,sentiment)
        newSentenceObj.setSentenceTokens(val)
        finalSentences.append(newSentenceObj)
    
    sentenceObj.addFinalSentences(finalSentences)   
    return sentenceObj

In [48]:
def trimSuggestions(sentenceObj, mainWord, listOfAlternatives):
    mainSentiment = senty.polarity_scores(mainWord)['compound']
    isPositive = True if(mainSentiment > 0) else False
    finalWords = []
    for word in listOfAlternatives:
        if(word) in BANNED_WORDS: continue
        senty_suggestion = senty.polarity_scores(word)['compound'];
        isPositive_suggestion = True if senty_suggestion > 0 else False
        if(KEEP_SENSE and isPositive_suggestion != isPositive): continue
        if(senty_suggestion == 0):
            continue
        else:
            finalWords.append(word)
    if(len(finalWords) == 1):
        if(finalWords[0] == mainWord): return []
    return finalWords

def getAlternativeWords(sentenceObj):
    mainSentence = sentenceObj.ogSentence;
    mainSentiment = sentenceObj.ogSentiment;

    sentenceTokens = cleanAndTokenizeText(mainSentence)
    sentenceObj.setSentenceTokens(sentenceTokens)

    numberOfSentimentHeavyWords = 0;
    for ind, word in enumerate(sentenceTokens):
        alternativeSentenceWithHTML = []
        copyOfTokens = sentenceTokens[:]
        replacements = []
        replacements = sentenceObj.checkIfWordExists(word)
        if(replacements != []):
            print("FOUND REPEATED WORD {0}".format(word))
            sentenceObj.addAlternativesByIndex(ind, replacements)
            continue;
        
        if(HANDLE_NEGATIONS):
            if(word in LIST_OF_NEGATIONS):
                
                '''
                    If the word is in the list of negations
                    We add a [IGNORE_FLAG] tag to the alternatives
                    Which will be grabbed in the future by the sentence generation algorithm. (chunking)
                '''
                
                tmp = ' '.join(sentenceTokens[ind:ind+3])
                if(senty.polarity_scores(tmp)['compound'] != 0):
                    sentenceObj.addAlternativesByIndex(ind, ["IGNORE_FLAG", word]);
                    continue

        score = senty.polarity_scores(word)['compound']
        if(score != 0.0):
            if(numberOfSentimentHeavyWords < MAX_SENTIMENT_HEAVY_WORDS):
                numberOfSentimentHeavyWords+=1;
            else:
                print("Sentiment heavy words:  {0}/{1}".format(numberOfSentimentHeavyWords, MAX_SENTIMENT_HEAVY_WORDS))
                return sentenceObj
            
            tempReplacements = returnReplacementsForWord(word) # get embedding based relations
            
            if(VERBOSE_PRINTING): print("Some early replacements for word {0} are {1}".format(word, tempReplacements[0:8]))
            if(tempReplacements == []):
                print("No replacements found at all for word {0}".format(word))
                continue
            
            replacements = posApprovedReplacements(tempReplacements[:], copyOfTokens[:], ind)
            finalReplacements = trimSuggestions(sentenceObj,word, replacements)
            
            if(finalReplacements == []):
                if(VERBOSE_PRINTING): print(" -- No POS approved words after trimming! -- for word {0}\n some non-trimmed:{1}".format(word, tempReplacements[:4]))
                continue
            
            # Trimming until max replacements to ensure efficiency
            finalReplacements = finalReplacements[:MAX_REPLACEMENTS]
            sentenceObj.addAlternativesByIndex(ind, finalReplacements)
            sentenceObj.addWordToAlternatives(word, finalReplacements)
    print("Sentiment heavy words:  {0}/{1}".format(numberOfSentimentHeavyWords, MAX_SENTIMENT_HEAVY_WORDS))
    return sentenceObj




In [49]:
def interactionHelper(dictOfAltObjs, sentiments, thisSent, lastWorkingIndex=0):
    if(thisSent in sentiments):
        mainIndex = sentiments.index(thisSent)
        
    else:
        mainIndex = lastWorkingIndex
    
    numLower =  mainIndex  # Mod to keep it in range 0 -> val 
    numHigher = len(sentiments) - mainIndex - 1

    print("There are totally {0} possible sentences".format(len(sentiments)))
    print("Current sentiment is at index {0}, and length of sentiments is {1}".format(mainIndex, len(sentiments)));
    print(" <--- {0} lower and {1} ---> higher ".format(numLower, numHigher))
    print("Would you like to see a lower sentiment or higher sentiment or ALL?")
    print("Please enter 1 for lower, 2 for higher or 0 for all")
    
    targetSentiment = thisSent
    textInput = input()
    if(textInput == ""):
        print("You entered nothing! exiting")
        return
        
    if(textInput.isdigit()):
        textInput = int(textInput)
        if(textInput == 0):
            if(VERBOSE_PRINTING): print("Printing sentiments from : {0}".format(sentiments))
            for sentiment in sentiments:
                obj = dictOfAltObjs[sentiment][0];
                displayText("{0}  : {1}".format(obj.getHTML(), sentiment))
            return
        elif(textInput == 1 and numLower > 0 and (mainIndex - 1) >= 0):
            targetSentiment = sentiments[mainIndex - 1]
            htmlSentence = dictOfAltObjs[targetSentiment][0].getHTML();
            displayText("{0} : {1}".format(htmlSentence, targetSentiment))
            
        elif(textInput == 2 and numHigher < len(sentiments) and (mainIndex + 1) < len(sentiments)):
            targetSentiment = sentiments[mainIndex + 1]
            htmlSentence = dictOfAltObjs[targetSentiment][0].getHTML();
            displayText("{0} : {1}".format(htmlSentence, targetSentiment))
            
        else:
            print("You entered an invalid option")
            interactionHelper(dictOfAltObjs, sentiments, thisSent, mainIndex)
            return
        
    interactionHelper(dictOfAltObjs, sentiments, targetSentiment,mainIndex)



def interactionWithUser(mainSentenceObj, dictOfAltObjs,sentiments, isUser=False):
    if(isUser == False):
        return
    else:
        mainSentence = mainSentenceObj.ogSentence;
        mainSent = mainSentenceObj.ogSentiment;
        if(VERBOSE_PRINTING):
            print("The following are the sentiment values for the input")
            for ind,sent in enumerate(sentiments):
                print("{0} - {1}".format(ind+1,sent))
        
        interactionHelper(dictOfAltObjs, sentiments, mainSent) 

In [50]:
def extractTwitterDataset():
#     df_tweets = pd.read_csv( pathToDatasets + 'cleanedTweets.csv', nrows=NUM_OF_TWEETS, skiprows =TWEET_START)
    df_tweets = pd.read_csv( pathToDatasets + "{0}.csv".format(DATASET_FILE_NAME), nrows=NUM_OF_TWEETS, skiprows =TWEET_START)
    tweets = df_tweets.values
    return tweets;

def createDictionaryOfSentStrings(sentencesWithSentiment):
    dict_sentimentToStringObjects = {}
    for obj in sentencesWithSentiment:
        sent = obj.getSentiment()
        if(dict_sentimentToStringObjects.get(sent) == None):
            dict_sentimentToStringObjects[sent] = [obj]
        else:
            (dict_sentimentToStringObjects[sent]).append(obj)

    return dict_sentimentToStringObjects;

def runThroughTweets():
    tweets = extractTwitterDataset()
    
    counterOfTweets = 0;
    for counter,tweet in enumerate(tweets):
        counterOfTweets +=1;
        tweet = tweet[0]
        tweetTokens = cleanAndTokenizeText(tweet)
        mainSentiment = senty.polarity_scores(tweet)['compound']
        if(mainSentiment == 0):
            continue
        print("\n\n {2} Tweet: {0}:{1}\n".format(tweet,mainSentiment, counterOfTweets))   
        
        sentenceObj = Sentence(tweet, mainSentiment)
        sentenceObj = getAlternativeWords(sentenceObj)
        sentenceObj = editBoosterWords(sentenceObj)
        
        replacementDictionary = sentenceObj.getDictOfIndexWords();
        if(len(replacementDictionary) <= 0):        
            print(" -- No new Strings generated ---\n\n")
            continue

        keysToChange = replacementDictionary.keys();
        if(VERBOSE_PRINTING):
            for key in keysToChange:
                if(key not in range(0,len(tweetTokens))): continue
                print("{0}'th word's ({2}) options: {1}".format(key,replacementDictionary[key],tweetTokens[key]))

        sentenceObj = returnCombinationsOfStrings(sentenceObj)
        sentencesWithSentiment, sentenceObj = printStrings(sentenceObj, counterOfTweets)
       
        dict_sentimentToStringObjects = createDictionaryOfSentStrings(sentencesWithSentiment)
        
        sentiments = sorted(dict_sentimentToStringObjects.keys())
        
    return counterOfTweets;
        

numOfTweets = runThroughTweets()
print("num of tweets done : {0}".format(numOfTweets))

global LIST_OF_EVAL_OBJECTS

with open("eval.json", "w+") as evalWriter:
        jsonString = json.dumps([obj.__dict__ for obj in LIST_OF_EVAL_OBJECTS])
        evalWriter.write(jsonString)
print("Eval writer written!")



 1 Tweet: Job Interview in Cardiff today wish me luck! Got about 3 hours sleep :0.7177

Some synonyms for word wish are ['indirect_request', 'wishing', 'regard']
Some early replacements for word wish are ['hope', 'remember', 'forget', 'have', 'you', 'better', 'think', 'always']
No antonyms found for word luck
Some synonyms for word luck are ['chance', 'lot', 'luck']
Some early replacements for word luck are ['goodluck', 'hope', 'good', 'congrats', 'hopefully', 'thanks', 'well', 'thank']
Sentiment heavy words:  2/6
5'th word's (wish) options: {'wish', 'hope', 'care', 'want'}
7'th word's (luck) options: {'chance', 'luck'}
Number of negations removed : 0
--- More options (total: 15) possible ---
Actual Sentence: Job Interview in Cardiff today wish me luck! Got about 3 hours sleep  :0.7177
Worst Sentence: Job Interview in Cardiff today want me chance ! Got about 3 hours sleep : 0.3802
Best Sentence: Job Interview in Cardiff today care me luck ! Got about 3 hours sleep : 0.7574
Wrote some

Sentiment heavy words:  1/6
3'th word's (want) options: {'desire', 'wish', 'want'}
2'th word's (I) options: {'BOOSTER_FLAG'}
Number of negations removed : 0
--- More options (total: 32) possible ---
Actual Sentence:  I think I want to read some books but the library does not have them  :0.0387
Worst Sentence: I think I want to read some books but the library does not have them : 0.0387
Best Sentence: I think I definitely wish to read some books but the library does not have them : 0.4019
Wrote something!


 10 Tweet: My nap was interrupted so many times today  Going out for Japanese with the  arents again...:-0.296

No antonyms found for word interrupted
Some synonyms for word interrupted are ['disrupt', 'break', 'cut_off']
Some early replacements for word interrupted are ['interrupting', 'rudely', 'interrupts', 'interrupt', 'interupted', 'woken', 'lectured', 'questioned']
Sentiment heavy words:  1/6
3'th word's (interrupted) options: {'interrupted', 'disturb', 'questioned'}
Number of 

IndexError: list index out of range

## Testing Code


In [ ]:
specificWord = "good"
def testOneWord(word=""):
    if(word==""):
        return
    print(word)

In [ ]:
from IPython.core.debugger import set_trace


specificString = ""
def specificString(textString=""):
    if(textString == "" or textString == None):
        return
    textString = dataClean(textString)
    textTokens = cleanAndTokenizeText(textString)
    mainSentiment = senty.polarity_scores(textString)['compound']
    if(mainSentiment == 0):
        print("{} \n No sentiment found in sentence".format(textString));
        return;
    print("\n {0}:{1}\n".format(textString,mainSentiment))   
    sentenceObj = Sentence(textString, mainSentiment)
    sentenceObj = getAlternativeWords(sentenceObj)
    sentenceObj = editBoosterWords(sentenceObj)
    
    
    replacementDictionary = sentenceObj.getDictOfIndexWords();
    if(len(replacementDictionary) <= 0):        
        print(" -- No new Strings generated ---\n\n")
        return
    
    keysToChange = replacementDictionary.keys();
    if(VERBOSE_PRINTING):
        for key in keysToChange:
               print("{0}'th word's ({2}) options: {1}".format(key,replacementDictionary[key],textTokens[key]))
    
    sentenceObj = returnCombinationsOfStrings(sentenceObj)
    allPossibleSentences, sentenceObj = printStrings(sentenceObj)
    
print("Enter a string to use")


inputText = ""
inputText = input()
if(inputText == ""):
    return
elif(inputText == "t"):
#     specificString("hot chocolate really angers me and it should sadden you too")
#     specificString("the grind is inspirational and saddening at the same time. do not want you to stop cuz i like what u do!")
    specificString("i hope they will increase the capacity fast yesterday was such a pain. got the fail whale 15 times in 2 hours....");
#     specificString("I would love to post the exec summary I raised >$3M with in first round, but its quite frankly a fucking embarrassment. I feel bad for the investors. actually I dont, because well, fuck them. wasnt their money anyway.")
else:
    specificString(inputText)
# I really love hot chocolate, but I'm not good with hot coffee 